In [1]:
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
from torchtext.vocab import GloVe

TEXT = data.Field(lower=True, fix_length=50, batch_first=True)
LABEL = data.Field(sequential=False,)

train, dev, test = data.TabularDataset.splits(
    path='SST-2', train='train.tsv', validation='dev.tsv',
    test='test.tsv', format='tsv', skip_header=True,
    fields=[('text', TEXT), ('label', LABEL)])
print("the size of train: {}, dev:{}, test:{}".format(len(train.examples), len(dev.examples), len(test.examples)))

TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=50), max_size=25000)
LABEL.build_vocab(train,)

print("train.fields:", train.fields, TEXT.vocab.vectors.shape)

train_iter, dev_iter, test_iter = data.BucketIterator.splits(
        (train, dev, test), batch_sizes=(64, 64, 64), sort_key=lambda x: len(x.text), sort_within_batch=True, repeat=False
    )
train_iter.repeat = False
test_iter.repeat = False

the size of train: 65328, dev:872, test:2021
train.fields: {'text': <torchtext.data.field.Field object at 0x00000283AD246F10>, 'label': <torchtext.data.field.Field object at 0x00000283AD246EB0>} torch.Size([14795, 50])


In [2]:
# Text-CNN Parameter
sequence_length = 50
vocab_size = TEXT.vocab.vectors.shape[0]
embedding_size = TEXT.vocab.vectors.shape[1]
num_classes = 2  # 0 or 1
filter_sizes = [2, 3, 5] # n-gram window
num_filters = 2
lr = 1e-3

In [3]:
import tenseal as ts

# Create TenSEAL context
context_client = ts.context(
    ts.SCHEME_TYPE.CKKS, 16384, coeff_mod_bit_sizes=[ 58, 40, 40, 40, 40, 40, 40, 40, 40, 58 ]
)
# set the scale
context_client.global_scale = pow(2, 40)
# generated galois keys in order to do rotation on ciphertext vectors
context_client.generate_galois_keys()

In [4]:
import time

class Sigmoid(nn.Module):
    def __init__(self):
        super(Sigmoid, self).__init__()
 
    def forward(self, x):
        x = 0.5 + 0.197*x - 0.004*torch.pow(x, 3)
        return x

class Softmax(nn.Module):
    def __init__(self):
        super(Softmax, self).__init__()
 
    def forward(self, x):
        x = 0.25 + 0.5*x + 0.125*torch.pow(x, 2)
        return x

class Swish(nn.Module):
    def __init__(self):
        super(Swish, self).__init__()
 
    def forward(self, x):
        x = 0.1198 + 0.5*x + 0.1473*torch.pow(x, 2) - 0.002012*torch.pow(x, 4)
        return x

def sigmoid(ckks_vec):
    return ckks_vec.polyval([0.5, 0.197, 0, -0.004])
    
def softmax(ckks_vec):
    return ckks_vec.polyval([0.25, 0.5, 0.125])

def swish(ckks_vec):
    return ckks_vec.polyval([0.1198, 0.5, 0.1473, 0, -0.002012])

class HETextCNN(nn.Module):
    def __init__(self):
        super(HETextCNN, self).__init__()

        self.num_filters_total = num_filters * len(filter_sizes)
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        
        self.convs = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(1, num_filters, (kernel, embedding_size), bias=False),
                Swish(),
                nn.AvgPool2d((sequence_length - kernel + 1,1))
            ) for kernel in filter_sizes])
        
        self.fc = nn.Linear(self.num_filters_total,num_classes)
        self.sm = Softmax()
                           
    def forward(self, X):
        embedded_chars = self.embedding(X)# [batch_size, sequence_length, embedding_size]
        embedded_chars = embedded_chars.unsqueeze(1)

        # Plain forward
        plain_out = [conv(embedded_chars) for conv in self.convs]
        plain_out = torch.cat(plain_out, dim=1)
        plain_out = plain_out.view(embedded_chars.size(0), -1)
        plain_out = self.fc(plain_out)
        print(plain_out)
        plain_logit = self.sm(plain_out)
        
        # Cipher forward
        cipher_logits = []
        for single in embedded_chars:
            # Encrypt and encode
            x_enc1, windows_nb1 = ts.im2col_encoding(context_client, single[0][:26].tolist(), 2, embedding_size, 1)
            x_enc2, windows_nb2 = ts.im2col_encoding(context_client, single[0][25:].tolist(), 2, embedding_size, 1)

            x_enc3, windows_nb3 = ts.im2col_encoding(context_client, single[0][:27].tolist(), 3, embedding_size, 1)
            x_enc4, windows_nb4 = ts.im2col_encoding(context_client, single[0][25:].tolist(), 3, embedding_size, 1)

            x_enc5, windows_nb5 = ts.im2col_encoding(context_client, single[0][:27].tolist(), 5, embedding_size, 1)
            x_enc6, windows_nb6 = ts.im2col_encoding(context_client, single[0][23:].tolist(), 5, embedding_size, 1)

            cipher_out = []
            fc_weight = self.fc._parameters['weight'].clone().T

            for idx in range(len(self.convs)):
                conv_weights = self.convs[idx][0]._parameters['weight'].tolist()
                for channel in range(num_filters):
                    kernel = conv_weights[channel][0]

                    if len(kernel) == 2: # 2-gram, 25+24
                        c_conv1 = x_enc1.conv2d_im2col(kernel, windows_nb1)
                        c_conv2 = x_enc2.conv2d_im2col(kernel, windows_nb2)
                        
                        h1 = swish(c_conv1)
                        h2 = swish(c_conv2)
                        ap = h1.sum() + h2.sum() # [1, sequence_length - filter_size + 1]

                        cipher_out.append(ap)
                        print("Conv out: ", ap.decrypt())
                    elif len(kernel) == 3: # 3-gram, 25+23
                        c_conv1 = x_enc3.conv2d_im2col(kernel, windows_nb3)
                        c_conv2 = x_enc4.conv2d_im2col(kernel, windows_nb4)

                        h1 = swish(c_conv1)
                        h2 = swish(c_conv2)
                        ap = h1.sum() + h2.sum() # [1, sequence_length - filter_size + 1]

                        cipher_out.append(ap)
                        print("Conv out: ", ap.decrypt())
                    elif len(kernel) == 5: # 5-gram, 23+23
                        c_conv1 = x_enc5.conv2d_im2col(kernel, windows_nb5)
                        c_conv2 = x_enc6.conv2d_im2col(kernel, windows_nb6)

                        h1 = swish(c_conv1)
                        h2 = swish(c_conv2)
                        ap = h1.sum() + h2.sum() # [1, sequence_length - filter_size + 1]

                        cipher_out.append(ap)
                        print("Conv out: ", ap.decrypt())

                    fc_weight[idx * num_filters + channel] /= sequence_length - len(kernel) + 1

            cipher_out = ts.pack_vectors(cipher_out)

            fc_bias = self.fc._parameters['bias']
            cipher_out = cipher_out.mm_(fc_weight.tolist()) + fc_bias.tolist() # [1, num_classes]

            print("FC out: ", cipher_out.decrypt())
            cipher_logit = softmax(cipher_out)

            cipher_logits.append(cipher_logit.decrypt())
            print("Softmax out: ", cipher_logit.decrypt())
        
        acc_loss = (np.abs(np.array(cipher_logits) - np.array(plain_logit.tolist())) / np.array(plain_logit.tolist())).sum()
        
        print("Batch acc loss: ", acc_loss)
        
        return plain_logit
    
    def forward_plain(self, X):
        embedded_chars = self.embedding(X)# [batch_size, sequence_length, sequence_length]
        embedded_chars = embedded_chars.unsqueeze(1)

        out = [conv(embedded_chars) for conv in self.convs]
        out = torch.cat(out, dim=1)
        out = out.view(embedded_chars.size(0), -1)
        out = self.fc(out)
        logit = self.sm(out)
        return logit

In [5]:
def binary_acc(preds, y):
    correct = torch.eq(preds, y).float()
    acc = correct.sum() / len(correct)
    return acc

train_set_size = 128

def train(model, optimizer, criterion):
    avg_acc = []
    avg_loss = []
    model.train()
    for batch_idx , batch in enumerate(train_iter):
        if batch_idx >= train_set_size:
            continue
        text, labels = batch.text , batch.label - 1
        predicted = model.forward_plain(text)

        acc = binary_acc(torch.max(predicted, dim=1)[1], labels)
        avg_acc.append(acc)
        loss = criterion(predicted, labels)
        avg_loss.append(loss)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    return np.array(avg_acc).mean()

def evaluate(model, criterion):
    avg_acc = []
    model.eval()
    for batch_idx , batch in enumerate(dev_iter):
        text, labels = batch.text , batch.label - 1
        predicted = model(text)

        acc = binary_acc(torch.max(predicted, dim=1)[1], labels)
        avg_acc.append(acc)

    return np.array(avg_acc).mean()

In [6]:
model = HETextCNN()
print(model)

pretrained_embedding = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embedding)

optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

train_accs, test_accs = [], []

for epoch in range(50):

    train_acc = train(model, optimizer, criterion)
    print('epoch={},训练准确率={}'.format(epoch, train_acc))
test_acc = evaluate(model, criterion)
print("epoch={},测试准确率={}".format(epoch, test_acc))

HETextCNN(
  (embedding): Embedding(14795, 50)
  (convs): ModuleList(
    (0): Sequential(
      (0): Conv2d(1, 2, kernel_size=(2, 50), stride=(1, 1), bias=False)
      (1): Swish()
      (2): AvgPool2d(kernel_size=(49, 1), stride=(49, 1), padding=0)
    )
    (1): Sequential(
      (0): Conv2d(1, 2, kernel_size=(3, 50), stride=(1, 1), bias=False)
      (1): Swish()
      (2): AvgPool2d(kernel_size=(48, 1), stride=(48, 1), padding=0)
    )
    (2): Sequential(
      (0): Conv2d(1, 2, kernel_size=(5, 50), stride=(1, 1), bias=False)
      (1): Swish()
      (2): AvgPool2d(kernel_size=(46, 1), stride=(46, 1), padding=0)
    )
  )
  (fc): Linear(in_features=6, out_features=2, bias=True)
  (sm): Softmax()
)
epoch=0,训练准确率=0.5623779296875
epoch=1,训练准确率=0.6224365234375
epoch=2,训练准确率=0.685546875
epoch=3,训练准确率=0.7508544921875
epoch=4,训练准确率=0.8013916015625
epoch=5,训练准确率=0.820556640625
epoch=6,训练准确率=0.849365234375
epoch=7,训练准确率=0.8629150390625
epoch=8,训练准确率=0.87548828125
epoch=9,训练准确率=0.8793945312

Conv out:  [241.31054305925278]
Conv out:  [240.60921679208832]
FC out:  [10.989668859633522, 8.064369770967605]
Softmax out:  [20.841675878769692, 12.411577827586537]
Conv out:  [15.299580379084098]
Conv out:  [79.04524312773368]
Conv out:  [260.7805638984521]
Conv out:  [206.8358611631728]
Conv out:  [259.9873287022399]
Conv out:  [258.06338379563107]
FC out:  [10.703336891990372, 9.227088023979746]
Softmax out:  [19.922073375957034, 15.506111242767014]
Conv out:  [27.8960715075199]
Conv out:  [87.48036986732536]
Conv out:  [247.14709316149563]
Conv out:  [192.0372571787078]
Conv out:  [263.3501185147921]
Conv out:  [265.31944708287347]
FC out:  [9.525391745434344, 10.304224352510236]
Softmax out:  [16.35451530661136, 18.67445406607104]
Conv out:  [10.846427719988965]
Conv out:  [67.99573023650994]
Conv out:  [261.74150102915206]
Conv out:  [208.8743915610586]
Conv out:  [247.70728568075458]
Conv out:  [237.8333465284514]
FC out:  [11.033250252130054, 8.144499661852915]
Softmax out: 

KeyboardInterrupt: 